# r/place Explorer

[Place](https://www.reddit.com/r/place/) is cool. But a [static image](https://i.redd.it/o4oku48qk9py.png) doesn't tell the story of [battles won](https://gfycat.com/InfamousShyEeve) or [characters evolving](http://i.imgur.com/KVXH8G4.png) over time, and it's tough to find and point out all the [amazing details](https://i.redd.it/glxdg7d23cpy.png), particularly ephemeral ones that were [overwritten](https://i.redd.it/qtvc4dai7soy.png).

That's why we need an interactive playable, rewindable, annotatable _place explorer_.

## An interactive guide

A decent [corpus of data exists](http://spacescience.tech/place/img/) comprised of individual frames. Our job is to convert that ~5gb of images into a manageable amount of data that can quickly be transferred to the client, then use that data to create an interactive canvas that can be played forward or backwards in time.

Once we have interactive dynamic display, we can add annotations. Annotations consist of a title ("Skeletor smokes a blunt"), a time window (start time & stop time) and a location window (two points defining a rectangle of interest).

Users can add new annotations, which are stored in a database. They can also upvote existing annotations.

## Acquiring and processing the data

Before we can do any of this we need to acquire the raw data and process it into a managable form.

The process here is to grab the initial image, then the next image and diff the two, storing any changed pixels in a list. This list describes how to efficiently get from one image to the next. All 23k images can be stored this way in a reasonable amount of space, and an HTML canvas should be able to handle updates like this just fine.

Below is some noodling on getting the data. So far we're grabbing and processing individual images and individual diffs. To do includes automatically downloading and processing _all_ of the images and writing the aggregate diffs to a file.

In [1]:
import scipy.misc
import numpy as np
import requests
from PIL import Image
from io import BytesIO

In [2]:
scipy.misc.imread

<function scipy.misc.pilutil.imread>

In [3]:
img1 = scipy.misc.imread('1.png').reshape(1000000, 3).tolist()
img2 = scipy.misc.imread('2.png').reshape(1000000, 3).tolist()

In [10]:
zipped = zip(img1, img2)
type(zipped)
delta = [(i, item[1]) for i, item in enumerate(zipped) if item[0] != item[1]]
delta[:10]

[(2993, [34, 34, 34]),
 (5073, [34, 34, 34]),
 (19442, [34, 34, 34]),
 (20981, [255, 255, 255]),
 (21900, [255, 167, 209]),
 (34103, [229, 0, 0]),
 (34108, [229, 0, 0]),
 (34237, [34, 34, 34]),
 (35003, [229, 0, 0]),
 (35168, [34, 34, 34])]

In [5]:
i1 = requests.get("http://spacescience.tech/place/img/1491209117.png")
i2 = requests.get("http://spacescience.tech/place/img/1491209126.png")

In [6]:
imgg1 = Image.open(BytesIO(i1.content))
imgg2 = Image.open(BytesIO(i2.content))

a = np.asarray(imgg1).reshape(1000000).tolist()
b = np.asarray(imgg2).reshape(1000000).tolist()

In [11]:
zapped = zip(a, b)
delt = [(i, item[1]) for i, item in enumerate(zapped) if item[0] != item[1]]
delt[10]

(35251, 0)

In [ ]:
# Get image file list.
# Get first image. Baseline.
# Get second image. Diff, write to file. Repeat.